In [4]:
import json
from scipy.spatial import Delaunay
import functools
from matplotlib.pyplot import *
from scipy import interpolate

import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate, signal, ndimage
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.cluster import KMeans #, SpectralClustering, AgglomerativeClustering
from sklearn.neighbors import NearestNeighbors
import networkx as nx
# import statsmodels.api as sm

import itertools
import operator
from scipy.signal import find_peaks
from tsmoothie.smoother import *
from fractions import Fraction

In [5]:
def butter_lowpass_filter(new_u, data, cutoff=0.1, order = 2):
	a, b = new_u[0], new_u[-1]
	timestep = (b - a)/(new_u.shape[0] - 1) # dt of discretization
	fs = 1.0/timestep # frequency of discretization
	nyq = 0.5*fs
	normal_cutoff = cutoff / (nyq * timestep)
	# Get the filter coefficients 
	b, a = butter(order, normal_cutoff, btype='low', analog=False)
	y = filtfilt(b, a, data)#, method="pad", padlen=5
	return y




def toX(x, y):
    return np.concatenate((x.reshape(-1,1), y.reshape(-1,1)), axis=1)

def fromX(X):
    return X[:,0], X[:,1]

def PCAsort(x, y):
    # PCA
    xy = toX(x, y)
    pca = PCA(2).fit(xy)
    xypca = pca.transform(xy)
    newx, newy = fromX(xypca)

    #sort
    indexSort = np.argsort(newx)
    return newx[indexSort], newy[indexSort], pca

def interp_data(x, y, N):
    f = interpolate.interp1d(x, y, kind='linear')
    newx = np.linspace(x[0], x[-1], N)
    return newx, f(newx)

def align_data(x, y, direction):
    ddir = np.hstack((x[-1] - x[0], y[-1] - y[0]));
    return (x, y) if np.dot(direction, ddir) > 0 else (x[::-1], y[::-1])


# Solution from https://stackoverflow.com/a/63368162/2531400
def XYclean(x, y, window=None, N=None):
    newx, newy, pca = PCAsort(x, y)
    newx, newy = interp_data(newx, newy, N)
    newy = signal.savgol_filter(newy, window, 2)
    xyclean = pca.inverse_transform(toX(newx, newy))
    return fromX(xyclean)



def alpha_shape(points, alpha, only_outer=True):
    """
    Compute the alpha shape (concave hull) of a set of points.
    :param points: np.array of shape (n,2) points.
    :param alpha: alpha value.
    :param only_outer: boolean value to specify if we keep only the outer border
    or also inner edges.
    :return: set of (i,j) pairs representing edges of the alpha-shape. (i,j) are
    the indices in the points array.
    """
    assert points.shape[0] > 3, "Need at least four points"

    def add_edge(edges, i, j):
        """
        Add an edge between the i-th and j-th points,
        if not in the list already
        """
        if (i, j) in edges or (j, i) in edges:
            # already added
            assert (j, i) in edges, "Can't go twice over same directed edge right?"
            if only_outer:
                # if both neighboring triangles are in shape, it's not a boundary edge
                edges.remove((j, i))
            return
        edges.add((i, j))

    tri = Delaunay(points)
    edges = set()
    # Loop over triangles:
    # ia, ib, ic = indices of corner points of the triangle
    for ia, ib, ic in tri.vertices:
        pa = points[ia]
        pb = points[ib]
        pc = points[ic]
        # Computing radius of triangle circumcircle
        # www.mathalino.com/reviewer/derivation-of-formulas/derivation-of-formula-for-radius-of-circumcircle
        a = np.sqrt((pa[0] - pb[0]) ** 2 + (pa[1] - pb[1]) ** 2)
        b = np.sqrt((pb[0] - pc[0]) ** 2 + (pb[1] - pc[1]) ** 2)
        c = np.sqrt((pc[0] - pa[0]) ** 2 + (pc[1] - pa[1]) ** 2)
        s = (a + b + c) / 2.0
        area = np.sqrt(s * (s - a) * (s - b) * (s - c))
        circum_r = a * b * c / (4.0 * area)
        if circum_r < alpha:
            add_edge(edges, ia, ib)
            add_edge(edges, ib, ic)
            add_edge(edges, ic, ia)
    return edges

def find_min_max_curve(points, alpha, p0, pN):
    # Computing the alpha shape
    edges = alpha_shape(points, alpha=alpha, only_outer=True)
    #order edges
    edges = stitch_boundaries(edges)
    
    edge_points = np.zeros((len(edges),2))
    k=0
    for i, j in edges:
        edge_points[k,:] = points[[i, j], 0][0] , points[[i, j], 1][0]
        k += 1
    inodes, jnodes = zip(*edges)
    min_x_ind = np.argmin(np.linalg.norm(edge_points - p0,axis=1))
    max_x_ind = np.argmin(np.linalg.norm(edge_points - pN,axis=1))
    print("min_x_ind={} max_x_ind={}".format(min_x_ind, max_x_ind))
#     min_x_ind = np.argmin(edge_points[:, 0])
#     max_x_ind = np.argmax(edge_points[:, 0])
    if min_x_ind < max_x_ind:
        lower_hull = edge_points[min_x_ind:max_x_ind+1, :]
        upper_hull = np.concatenate([edge_points[max_x_ind:, :], edge_points[:min_x_ind+1, :]])
    else:
        upper_hull = edge_points[max_x_ind:min_x_ind+1, :]
        lower_hull = np.concatenate([edge_points[min_x_ind:, :], edge_points[:max_x_ind+1, :]])
    return lower_hull, upper_hull


def find_edges_with(i, edge_set):
    i_first = [j for (x,j) in edge_set if x==i]
    i_second = [j for (j,x) in edge_set if x==i]
    return i_first,i_second

def stitch_boundaries(edges):
    edge_set = edges.copy()
    boundary_lst = []
    while len(edge_set) > 0:
        boundary = []
        edge0 = edge_set.pop()
        boundary.append(edge0)
        last_edge = edge0
        while len(edge_set) > 0:
            i,j = last_edge
            j_first, j_second = find_edges_with(j, edge_set)
            if j_first:
                edge_set.remove((j, j_first[0]))
                edge_with_j = (j, j_first[0])
                boundary.append(edge_with_j)
                last_edge = edge_with_j
            elif j_second:
                edge_set.remove((j_second[0], j))
                edge_with_j = (j, j_second[0])  # flip edge rep
                boundary.append(edge_with_j)
                last_edge = edge_with_j

            if edge0[0] == last_edge[1]:
                break

        boundary_lst.append(boundary)
    return boundary_lst[0]

from matplotlib.pyplot import *
from scipy import interpolate
def find_first_peak(x_fil, y_fil, x0, xmin, xmax, x_mean, time):
	if x_fil[0] > x_fil[-1]:
		x_fil = x_fil[::-1]
		y_fil = y_fil[::-1]
	ind = np.argmax(y_fil) 
	print("max ={} {}".format(x_fil[ind], y_fil[ind]))    
	#choose some points if they are:
	args = (y_fil >= 0.3) & (x_fil <= x_mean)
	x_ripple = x_fil[args]
	y_ripple = y_fil[args]

	print("sizes of ripple:{} {}".format(x_ripple.shape, y_ripple.shape))
	#return 0, 0, 0, 0, 0, 0, 0
	#find the first peak in a smoothed curve
	peaks, props = find_peaks(y_ripple, prominence=0.001)
	print("peaks, props:", peaks, props)
	try:
		x_peak, y_peak = x_ripple[peaks[0]], y_ripple[peaks[0]]
	except:
		x_peak, y_peak = np.inf, np.inf
	length_x_clip = x_mean - x_peak
	print('x_peak candidates=', x_ripple[peaks], 'y_peak candidates=', y_ripple[peaks])
	print('x_peak', x_peak, 'y_peak=', y_peak)

	# calculate actual min max thickness and averaged min max values of thickness
	args = (x_fil >= x_peak) & (x_fil <= x_mean)
	print("yargs[args]=", y_fil[args])
	y_ripple_slice = y_fil[args]
	delta_min = 0.5 - y_ripple_slice.max()
	delta_max = 0.5 - y_ripple_slice.min()
	delta_min_smooth = 0.5 - y_ripple.max()
	delta_max_smooth = 0.5 - y_ripple.min()
	return x_peak, y_peak, length_x_clip, delta_min, delta_max, delta_min_smooth, delta_max_smooth


def find_smooth_curve_and_bounds(x, y, xcm, alpha = 0.1, N = 4000, M = 3, Ninterp = 4000, Npolyfit = 50, savgol_order = 2, window = 100,\
                        is_cycle = False, show_in_pca = False, plot_clusters=True, plot_all_points=True, \
                        plot_rhomb = True, show_cluster_num = True, show_split_line=True):
    Npca = N//M
    window += 1 if window % 2 == 0 else 0
    
    xmin = x.min()
    xmax = x.max()
    xcm = 0.5*(xmax + xmin)
    inds = x < xcm
    ind_xy0 = y[inds].argmin()
    xy0 = x[inds][ind_xy0], y[inds][ind_xy0]
#     print('xyo****=', xy0, y[inds])
    inds = x > xcm
    ind_xyN = y[inds].argmin()
    xyN = x[inds][ind_xyN], y[inds][ind_xyN]
    print('xy0=', xy0, 'xyN=', xyN)


    #split points into 3 parts
    cut_x1, cut_x2 = xmin + .08*(xmax-xmin), xmin + .7*(xmax-xmin)
    
    dcut = 0.05
    xmm = {
        0: (-np.inf, cut_x1),
        1: (cut_x1, cut_x2),
        2: (cut_x2, np.inf),
    }
    ymm = np.min(y) + 0.01*(np.max(y) - np.min(y)) # to cut edges of the fitting curve
    inds = [(x < cut_x1 + dcut*(xmax-xmin)),
            (x > cut_x1 - dcut*(xmax-xmin)) & (x < cut_x2 + dcut*(xmax-xmin)),
            (x >= cut_x2 - dcut*(xmax-xmin))]

    ccenters = np.zeros((M, 2))
    color = plt.cm.rainbow(np.linspace(0, 1, M))
    for i, c in zip(range(M), color):
        x_, y_ = np.asarray(x[inds[i]]), np.asarray(y[inds[i]])
        ccenters[i,:] = x_.mean(), y_.mean()
       

    # 3. Hybrid solution: clusterization + 2-NN + PCA
    # Sort clusters: https://stackoverflow.com/a/37744549/2531400
    clf = NearestNeighbors(n_neighbors=2).fit(ccenters)
    G = clf.kneighbors_graph(mode='distance')

    T = nx.from_scipy_sparse_matrix(G)
    i0 = 0  # number of the first cluster
    if not is_cycle:
        min_dist = np.inf
        for i in range(M):
            dist = np.linalg.norm(ccenters[i] - xy0)
            if dist < min_dist:
                i0 = i; min_dist = dist

    order = list(nx.dfs_preorder_nodes(T, i0))

    X, Y = [], []
    for i in range(M):
        j = order[i]
        x_, y_ = np.asarray(x[inds[j]]), np.asarray(y[inds[j]])
        # x_, y_ = x[kmeans.labels_ == j], y[kmeans.labels_ == j]
        x_, y_, pca = PCAsort(x_, y_)
        x_, y_ = fromX(pca.inverse_transform(toX(x_, y_)))
        if i < M-1:
            x_, y_ = align_data(x_, y_, ccenters[order[i+1]] - ccenters[j])
        else:
            x_, y_ = align_data(x_, y_, ccenters[j] - ccenters[order[i-1]])
            #print(ccenters[j], ccenters[order[i-1]])
        X.append(x_); Y.append(y_)

    XX, YY = np.array([]), np.array([])
    for i, c in zip(range(M), color):
        x_, y_ = X[i], Y[i]
        pca = PCA(2) #, whiten=True)
        x_, y_ = fromX(pca.fit_transform(toX(x_, y_)))

        xx = np.array([np.min(x_), np.max(x_), np.max(x_), np.min(x_), np.min(x_)])
        yy = np.array([np.min(y_), np.min(y_), np.max(y_), np.max(y_), np.min(y_)])

        print(f"Cluster {i} contains {X[i].size} points within \
            {np.min(x_):.2}<x<{np.max(x_):.2} & {np.min(y_):.2}<y<{np.max(y_):.2}, \
            ratio = {(np.max(x_)-np.min(x_))/(np.max(y_)-np.min(y_)):.3}")
        
        #'''Chebyshev polynomial fit
        p = np.polynomial.Chebyshev.fit(x_, y_, Npolyfit)
        x_ = np.linspace(x_[0], x_[-1], Ninterp//M); y_ = p(x_)
        #'''
        x_, y_ = fromX(pca.inverse_transform(toX(x_, y_)))
        xx, yy = fromX(pca.inverse_transform(toX(xx, yy)))

        mask = (x_ > xmm[i][0]) & (x_ < xmm[i][1]) & (y_ > ymm)
        x_, y_ = x_[mask], y_[mask]
        # Fill 1-D arrays
        XX = np.hstack((XX, x_))
        YY = np.hstack((YY, y_))


    # upper and lower lines
    lower_hull, upper_hull = find_min_max_curve(np.c_[x,y], alpha = alpha, p0=xy0, pN=xyN)

    x_peak, y_peak, length_x_clip, delta_min, delta_max, delta_min_smooth, delta_max_smooth = find_first_peak(upper_hull[:,0], upper_hull[:,1], xy0[0], xmin, xmax, xcm, 0)
    return lower_hull, upper_hull, XX, YY, x_peak, y_peak, length_x_clip, delta_min, delta_max, delta_min_smooth, delta_max_smooth, xy0, xyN




import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from pathlib import Path
# pio.orca.config.executable = '/usr/bin/orca'
def plot_graph(list_x, list_y, names, xtitle, ytitle, image_name, list_x_fill=[], list_y_fill=[], mode=[],\
			   dash=['solid', 'dot', 'dash', 'longdash'],\
			   colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ],\
			   marker_size=15, xrange =[], yrange = [], \
			   marker_style = ['circle', 'triangle-up', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ],\
			   width=1000, height=500, path='./'):
	if mode == []:
		for i in range(len(list_x)):
			mode.append('lines+markers')

	figborderlinesize = 0.7
	legborderlinesize = 0.7
	yaxis = dict(
		tickfont = dict(
			family = 'Times New Roman',
			size = 20,
			color = 'black'
		),
		titlefont = dict(
			family = 'Times New Roman',
			size = 25,
			color = 'black'
		),
	)
	xaxis = dict(
		tickfont = dict(
			family = 'Times New Roman',
			size = 20,
			color = 'black'
		),
		titlefont = dict(
			family = 'Times New Roman',
			size = 25,
			color = 'black'
		)
	)

	axis_style = dict(showline=True, gridwidth=1, gridcolor='lightgrey', linewidth=figborderlinesize, linecolor='black', mirror=True, ticks='outside', tickfont = dict(family = 'Times New Roman', size = 20, color = 'black'))
	bg_style = {'plot_bgcolor': 'rgba(255, 255, 255, 1)', 'paper_bgcolor': 'rgba(255, 255, 255, 1)',}


	fig = go.Figure()
	
	for i,x in enumerate(list_x):
		y = np.asarray(list_y[i])
		fig.add_trace(go.Scatter(x=x, y=y, name=names[i],
								 mode=mode[i],
								 marker=dict(
									 size=marker_size,
									 line=dict(
										 color=colors[i],
										 width=1
									 )
								 ),
								 marker_symbol=marker_style[i],
								 line=dict(color=colors[i], width=2, dash=dash[i]),
								 textfont=dict(
									 family="Times New Roman",
									 size=18,
									 color="LightSeaGreen")
								 ))
	
	if len(list_x_fill) == 2 and len(list_y_fill) == 2:
		fig.add_trace(go.Scatter(x=list_x_fill[0], y=list_y_fill[0], mode='lines', line_color='indigo')) # fill down to xaxis
		fig.add_trace(go.Scatter(x=list_x_fill[1], y=list_y_fill[1], mode='lines', line_color='indigo', fill='tonexty')) # fill to trace0 y
	
	fig.update_layout(
		width = width,
		height = height,
		xaxis_title=xtitle,
		yaxis_title=ytitle,
		yaxis = yaxis,
		xaxis = xaxis,
		showlegend=True
	)
	fig.update_layout(bg_style)
	fig.update_xaxes(axis_style)
	fig.update_yaxes(axis_style)
	fig.update_layout(legend=dict(
		# yanchor="bottom",
		# y=0.1,
		# xanchor="left",
		# x=1.01,
		bgcolor="White",
		bordercolor="Black",
		borderwidth=figborderlinesize
	))
	fig.update_layout(
		autosize=False,
		margin=dict(
			l=0,
			r=0,
			b=0,
			t=0,
			pad=0.1
		),
		#     paper_bgcolor="LightSteelBlue",
	)
	
	if len(xrange) == 2:
		fig.update_xaxes(range=xrange)
	if len(yrange) == 2:
		fig.update_yaxes(range=yrange)
	fig.show()
	fn = path + image_name
	pio.write_image(fig, str(Path(fn)))
	print("Successfully generated:", fn)

In [ ]:
# *********************** Find the x coordinate of the first peak x_peak ****************************
x_mean = 17

path = '/Users/weugene/basilisk/work/tube/res21/'
# fn = 'r_over_x_total_t=16.1542.csv'
fn = 'r_over_x_total_t=9.98087.csv'
# fn = 'r_over_x_total_t=9.989.csv'
with open(path + fn, 'r') as f:
    lists = json.load(f)
    x, y = np.array(lists[0]), np.array(lists[1])

xcm = 18.7
find_smooth_curve_and_bounds(x, y, xcm, alpha=0.05, plot_clusters=False, plot_rhomb=False, show_cluster_num=False, show_split_line=False)

lower_hull, upper_hull, XX, YY, x_peak, y_peak, length_x_clip, delta_min, delta_max, delta_min_smooth, delta_max_smooth, xy0, xyN =  find_smooth_curve_and_bounds(x, y, x_mean, alpha = 0.1, N = 4000, M = 3, Ninterp = 4000, Npolyfit = 50, savgol_order = 2, window = 100, \
                        is_cycle = False, show_in_pca = False, plot_clusters=False, plot_all_points=True, \
                        plot_rhomb = False, show_cluster_num = False, show_split_line=False)

ind = (YY > 0.05)
#XX = XX[ind]
#YY = YY[ind]
#XX = np.concatenate([ [xy0[0]], XX, [xyN[0]] ])
#YY = np.concatenate([ [xy0[1]], YY, [xyN[1]] ])
fn = "r_over_x_t={}.csv".format(0)

plot_graph([x, XX, [x_peak, x_peak], [x_mean, x_mean]], [y, YY, [0,0.5], [0,0.5]], \
		   ['points', "smoothed", 'first peak', 'center of mass'], \
		   #list_x_fill=[upper_hull[:,0], lower_hull[:,0]], list_y_fill=[upper_hull[:,1], lower_hull[:,1]], \
		   dash=['solid', 'solid', 'dot', 'dot'], \
		   xtitle="x", ytitle="r", image_name=fn[:-3]+'pdf', mode=['markers', 'lines+markers', 'lines', 'lines'], \
		   colors=['blue', 'red', 'black', 'black' ], yrange=[0,0.5], \
		   marker_size=1, width=1000, height=500, path=path)

xy0= (13.372207641601562, 0.0034832845763717937) xyN= (15.930323600769043, 0.0034832845763717937)
Cluster 0 contains 94364 points within             -0.15<x<0.38 & -0.044<y<0.14,             ratio = 2.93


/Users/weugene/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


Cluster 1 contains 353671 points within             -0.84<x<1.0 & -0.038<y<0.11,             ratio = 12.2
Cluster 2 contains 152911 points within             -0.46<x<0.52 & -0.037<y<0.23,             ratio = 3.71
